In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
# train=pd.read_csv('train.csv', parse_dates = ['Dates'])
# test=pd.read_csv('test.csv', parse_dates = ['Dates'])
data=pd.read_csv('data/combined_weather_crime_lunar_final.csv')
data.head()

,Unnamed: 0,DATE,CCN,STATION,NAME,PRCP,TMAX,TMIN,Year,Month,Date,Day,Illum
0,0,1/1/16,103,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,56,41,2016,1,1,6,0.60
1,1,1/2/16,93,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,46,31,2016,1,2,7,0.50
2,2,1/3/16,100,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,48,29,2016,1,3,1,0.41
3,3,1/4/16,77,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,53,29,2016,1,4,2,0.32
4,4,1/5/16,92,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,38,15,2016,1,5,3,0.23


In [2]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

,DATE,CCN,STATION,NAME,PRCP,TMAX,TMIN,Year,Month,Date,Day,Illum
0,1/1/16,103,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,56,41,2016,1,1,6,0.60
1,1/2/16,93,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,46,31,2016,1,2,7,0.50
2,1/3/16,100,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,48,29,2016,1,3,1,0.41
3,1/4/16,77,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,53,29,2016,1,4,2,0.32
4,1/5/16,92,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,38,15,2016,1,5,3,0.23


In [3]:
data.rename(columns={"CCN": "CRIME-COUNT"}, inplace=True)

In [4]:
data['bins'] = np.nan

In [5]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,PRCP,TMAX,TMIN,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,56,41,2016,1,1,6,0.60,NaN
1,1/2/16,93,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,46,31,2016,1,2,7,0.50,NaN
2,1/3/16,100,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,48,29,2016,1,3,1,0.41,NaN
3,1/4/16,77,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,53,29,2016,1,4,2,0.32,NaN
4,1/5/16,92,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,38,15,2016,1,5,3,0.23,NaN


In [12]:
data.loc[data['CRIME-COUNT']<=75, ['bins']] = 'VeryLow'


In [13]:
data.loc[(data['CRIME-COUNT']>=76) & (data['CRIME-COUNT']<=90), ['bins']] = 'Low'

In [14]:
data.loc[(data['CRIME-COUNT']>=91) & (data['CRIME-COUNT']<=105), ['bins']] = 'Medium'

In [15]:
data.loc[(data['CRIME-COUNT']>=106) & (data['CRIME-COUNT']<=120), ['bins']] = 'High'

In [16]:
data.loc[(data['CRIME-COUNT']>=121, ['bins'])] = 'VeryHigh'

In [17]:
data['bins'].value_counts()

Medium      410
Low         374
High        212
VeryLow     175
VeryHigh     78
Name: bins, dtype: int64

In [18]:
data['CRIME-COUNT'].describe()

count    1249.000000
mean       93.541233
std        17.111019
min        28.000000
25%        82.000000
50%        93.000000
75%       105.000000
max       146.000000
Name: CRIME-COUNT, dtype: float64

In [19]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,PRCP,TMAX,TMIN,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,56,41,2016,1,1,6,0.60,Medium
1,1/2/16,93,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,46,31,2016,1,2,7,0.50,Medium
2,1/3/16,100,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,48,29,2016,1,3,1,0.41,Medium
3,1/4/16,77,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,53,29,2016,1,4,2,0.32,Low
4,1/5/16,92,USC00186350,"NATIONAL ARBORETUM DC, MD US",0.0,38,15,2016,1,5,3,0.23,Medium


In [20]:
X = data.drop("bins", axis=1)
y = data["bins"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [23]:
# X_scaler = MinMaxScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [24]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [25]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)